In [181]:
# The code was removed by Watson Studio for sharing.

<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

**Note**: This notebook works correctly with kernel `Python 3.10.x` with pyspark 3.3.x.

<a id="setup"></a>
## 1. Setup

### 1.1 Install Watson OpenScale and WML packages

In [182]:
!pip install --upgrade ibm-watson-openscale --no-cache --user| tail -n 1

In [183]:
!pip install --upgrade ibm-watson-machine-learning --no-cache --user| tail -n 1

Note: Restart the kernel to assure the new libraries are being used.

### 1.2 Configure credentials

In [184]:
CLOUD_API_KEY = "xiriDO_Tw_oYQeixxugQygpxHNEIWoOYZ0An2USovB4r"
IAM_URL="https://iam.us-south.bluemix.net/oidc/token"

In [185]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": CLOUD_API_KEY
}

In [186]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.308'

In [187]:
wml_client.spaces.list(limit=10)

------------------------------------  -----------------------  ------------------------
ID                                    NAME                     CREATED
b22781ad-5eff-44b0-a36c-6765799d7133  Lukasz Deployment Space  2023-05-28T18:59:07.579Z
------------------------------------  -----------------------  ------------------------


,ID,NAME,CREATED
0,b22781ad-5eff-44b0-a36c-6765799d7133,Lukasz Deployment Space,2023-05-28T18:59:07.579Z


In [188]:
WML_SPACE_ID='b22781ad-5eff-44b0-a36c-6765799d7133' # use space id here
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

In [189]:
MODEL_NAME = "P16"

## 3. Subscriptions <a id="subscription"></a>

### 3.1 Configuring OS

In [190]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment this line if using IAM token to authenticate
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.31'

**Note**: Please re-run the above cell if it doesn't work the first time.

In [191]:
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}
DB_CREDENTIALS = None
KEEP_MY_INTERNAL_POSTGRES = True

In [192]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart e25969b8-316a-4515-b5ea-5895bbbd2c55


In [193]:
SERVICE_PROVIDER_NAME = "ServiceName"
SERVICE_PROVIDER_DESCRIPTION = "ServiceDescription"

In [194]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: 1cb6efa5-036e-49be-b3a6-99b1aacb8427


In [195]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WMLCredentialsCloud(
            apikey=CLOUD_API_KEY,      ## use `apikey=IAM_TOKEN` if using IAM_TOKEN to initiate client
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider c8cfedcd-09f3-4d2e-ab59-2803138229f9 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [196]:
# Retrieve the list of asset deployment details
asset_deployment_details_list = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id=WML_SPACE_ID).result['resources']

# Print the available deployment names
available_names = [asset['entity']['name'] for asset in asset_deployment_details_list]
print("Available deployment names:", available_names)


Available deployment names: ['model_08JUN deployment', 'model_h_10 deployment', 'H7 Prediction model', 'best_model deployment', 'model_h_10 deployment', 'best_model deployment', 'model_08JUN deployment', 'model_h_10 deployment', 'model_h_10 deployment', 'model_h_10 deployment']


In [197]:
asset_deployment_details_list = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id=WML_SPACE_ID).result['resources']

DEPLOYMENT_NAME = 'H7 Prediction model'  # Use the model name here

matching_deployments = [asset for asset in asset_deployment_details_list if asset['entity']['name'] == DEPLOYMENT_NAME]

if len(matching_deployments) > 0:
    asset_deployment_details = matching_deployments[0]  # Select the first matching deployment
else:
    raise ValueError('Deployment with name "{}" not found.'.format(DEPLOYMENT_NAME))

# Use the asset deployment details
print("Asset deployment details:", asset_deployment_details)


Asset deployment details: {'metadata': {'guid': '7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0', 'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0?space_id=b22781ad-5eff-44b0-a36c-6765799d7133', 'created_at': '2023-06-09T16:05:35.621Z', 'modified_at': '2023-06-09T16:05:35.621Z'}, 'entity': {'name': 'H7 Prediction model', 'type': 'online', 'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0/predictions'}, 'asset': {}, 'asset_properties': {}}}


In [198]:
deployment_uid = asset_deployment_details['metadata']['guid']
deployment_uid

'7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0'

In [199]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=WML_SPACE_ID)
model_asset_details_from_deployment

{'metadata': {'guid': '7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0?space_id=b22781ad-5eff-44b0-a36c-6765799d7133',
  'created_at': '2023-06-09T16:05:35.621Z',
  'modified_at': '2023-06-09T16:05:35.621Z'},
 'entity': {'name': 'H7 Prediction model',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0/predictions'},
  'asset': {'asset_id': 'ac6ac79d-4ad0-4058-ada1-7e941c343b69',
   'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/models/ac6ac79d-4ad0-4058-ada1-7e941c343b69?space_id=b22781ad-5eff-44b0-a36c-6765799d7133&version=2020-06-12',
   'name': 'P16',
   'asset_type': 'model',
   'created_at': '2023-06-09T15:22:19.904Z',
   'modified_at': '2023-06-09T15:22:24.969Z',
   'problem_type': 'binary',
   'input_data_type': 'structured'},
  'asset_properties': {'problem_type': 'binary',
   'model_type': 'wml-hybri

In [200]:
model_uid = model_asset_details_from_deployment['entity']['asset']['asset_id']
scoring_url = model_asset_details_from_deployment['entity']['scoring_endpoint']['url']
print(model_uid)
print(scoring_url)

ac6ac79d-4ad0-4058-ada1-7e941c343b69
https://us-south.ml.cloud.ibm.com/ml/v4/deployments/7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0/predictions


### 3.2 Subscribe the asset

In [201]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [202]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest

In [203]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.UNSTRUCTURED_TEXT,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['metadata']['url'],
            scoring_endpoint=ScoringEndpointRequest(url=scoring_url) # scoring model without shadow deployment
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='labels',
            probability_fields=['probability'],
            prediction_field='predictionLabel',
            feature_fields = ["preprocessed_texts"],
            categorical_fields = ["preprocessed_texts"],
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

'13c2cf68-6b36-4e0a-961e-7584c8f819cd'

In [204]:
wos_client.subscriptions.show()

ac6ac79d-4ad0-4058-ada1-7e941c343b69,P16,e25969b8-316a-4515-b5ea-5895bbbd2c55,7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0,H7 Prediction model,c8cfedcd-09f3-4d2e-ab59-2803138229f9,preparing,2023-06-09 20:31:23.542000+00:00,13c2cf68-6b36-4e0a-961e-7584c8f819cd
7af59133-9a51-4193-83e7-062c88e810be,IMDBClassificationExperimentDallas - P4 XGB Classifier,e25969b8-316a-4515-b5ea-5895bbbd2c55,c878f44c-2dd8-47b8-9b2f-c173365d4a13,IMDB Classification deployment,7acc5cf1-43c6-498a-9a66-c2827bfc0e69,active,2023-06-09 16:31:06.538000+00:00,156bb6d1-7302-498e-b4ed-c953676c8ef0
b7efdd8a-269d-45c1-a100-443518cd4642,Ham_Spam_Model,e25969b8-316a-4515-b5ea-5895bbbd2c55,999b0e41-bfc1-4cba-8f1a-cb74567eb739,Ham_Spam_Model deployment,53042c62-ac2a-47c0-bd58-4ddee28dd43f,active,2023-06-07 15:15:02.828000+00:00,b662cec5-fa98-4fcf-bb48-88cb8ca303c0
ec884afa-24c1-4aed-9967-2d53ab5af161,DKaluzaSpamDetector,e25969b8-316a-4515-b5ea-5895bbbd2c55,ce4c59b4-6877-44f4-94dc-5b6b8c961e87,DKaluzaSpamDetector deployment,6f8a1561-19e0-4cfb-81c5-aff23195f126,active,2023-05-24 08:36:42.214000+00:00,2756a69d-a3f4-4474-9f53-2c9130c40bbb
8678081b-79a4-4b22-94c8-04c80dce9245,Text Binary Classifier,e25969b8-316a-4515-b5ea-5895bbbd2c55,34e05d06-f82a-4ba0-9dda-07aef2ecc53a,Text Binary Classifier deployment,7c1db92e-dd74-4c2f-a33b-5f3ac59c799c,active,2023-05-05 08:35:17.385000+00:00,02152349-cb27-4c4d-8ae3-5b3e06d71fe6


In [205]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  828e3d1a-4b7d-496b-9c1c-a3063a5bd204


### 3.3 Get subscription

In [206]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

{'metadata': {'id': '13c2cf68-6b36-4e0a-961e-7584c8f819cd',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/9f662e92df19cc1abadb5782b2f5c041:e25969b8-316a-4515-b5ea-5895bbbd2c55:subscription:13c2cf68-6b36-4e0a-961e-7584c8f819cd',
  'url': '/v2/subscriptions/13c2cf68-6b36-4e0a-961e-7584c8f819cd',
  'created_at': '2023-06-09T20:31:23.542000Z',
  'created_by': 'IBMid-66600302EK'},
 'entity': {'data_mart_id': 'e25969b8-316a-4515-b5ea-5895bbbd2c55',
  'service_provider_id': 'c8cfedcd-09f3-4d2e-ab59-2803138229f9',
  'asset': {'asset_id': 'ac6ac79d-4ad0-4058-ada1-7e941c343b69',
   'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/models/ac6ac79d-4ad0-4058-ada1-7e941c343b69?space_id=b22781ad-5eff-44b0-a36c-6765799d7133&version=2020-06-12',
   'name': 'P16',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'unstructured_text'},
  'asset_properties': {'training_data_schema': {'type': 'struct',
    'fields': [{'metadata': {'columnInfo': {'columnLength': 64},
     

### 3.4 Score the model and get transaction-id

In [207]:
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

In [208]:
text = "Ash cant die and is a badass. I love how stupid it can get and be very good. In most horror movies and dealing with the dead once scratched, bit, or lose limbs the character most likely dies. Ash is on the line of Jason, Freddy, and Michael who cant die. Another film of excessive blood and gore. These movie are stupid good." 
payload = {"input_data": [{"fields": ["preprocessed_texts"], "values": [[text]]}]}

response = wml_client.deployments.score(deployment_uid,payload)
print(response)

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.9872646509121095, 0.012735349087890485]]]}]}


In [216]:
wos_client.data_sets.get_records_count(payload_data_set_id)

1

## 4. Explainability

### 4.1 Configure Explainability

In [210]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 7446ca68-afe2-454f-b2ad-c27963140203 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### 4.2 Get explanation for the transaction

In [214]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result

if len(pl_records_resp["records"]) > 0:
    scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
    print("Running explanations on scoring IDs: {}".format(scoring_ids))
    
    explanation_types = ["lime"]
    result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
    
    print(result)
else:
    print("No records found in the dataset.")


Running explanations on scoring IDs: ['9b9e23b22978e1b1b49cb707dc2da08d-1']
{
  "metadata": {
    "explanation_task_ids": [
      "cef775ea-24d9-4c08-aa1b-f9a4436b61e4"
    ],
    "created_by": "IBMid-66600302EK",
    "created_at": "2023-06-09T20:32:39.166395Z"
  }
}


In [222]:
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
print(wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id, subscription_id=subscription_id).result)

{
  "metadata": {
    "explanation_task_id": "cef775ea-24d9-4c08-aa1b-f9a4436b61e4",
    "created_by": "IBMid-66600302EK",
    "created_at": "2023-06-09T20:32:39.166395Z",
    "updated_at": "2023-06-09T20:33:06.383540Z"
  },
  "entity": {
    "status": {
      "state": "finished"
    },
    "asset": {
      "id": "ac6ac79d-4ad0-4058-ada1-7e941c343b69",
      "name": "P16",
      "input_data_type": "unstructured_text",
      "problem_type": "binary",
      "deployment": {
        "id": "7ec4e10b-b84f-4fe0-a8dc-f7b56c5d4ed0",
        "name": "H7 Prediction model"
      }
    },
    "input_features": [
      {
        "name": "preprocessed_texts",
        "value": "Ash cant die and is a badass. I love how stupid it can get and be very good. In most horror movies and dealing with the dead once scratched, bit, or lose limbs the character most likely dies. Ash is on the line of Jason, Freddy, and Michael who cant die. Another film of excessive blood and gore. These movie are stupid good."
  